In [13]:
# =========================================
# ANALISE DE EXTREMOS TERMICOS - PADRAO WMO/IPCC/FAO
# =========================================

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import genextreme
from sklearn.preprocessing import StandardScaler
from fancyimpute import IterativeSVD
import warnings
warnings.filterwarnings('ignore')

# === CONFIGURACOES ===
caminho_arquivo = "C:/Python/Waves/maputoo.xlsx"
caminho_saida = "C:/Users/Dell/OneDrive/Rstudio/CONGRESSOJOH/EXTREMOS"
os.makedirs(caminho_saida, exist_ok=True)

# === 1. Carregar dados ===
df = pd.read_excel(caminho_arquivo)
df.columns = ['YEAR', 'DIA_MES', 'T2M_MAX', 'T2M_MIN']
df['DIA_MES'] = pd.to_datetime(df['DIA_MES'])

def corrigir_ano(row):
    try:
        return row['DIA_MES'].replace(year=int(row['YEAR']))
    except ValueError:
        return row['DIA_MES'].replace(day=28, year=int(row['YEAR'])) if row['DIA_MES'].month == 2 and row['DIA_MES'].day == 29 else pd.NaT

df['DATA_COMPLETA'] = df.apply(corrigir_ano, axis=1)
df = df.dropna(subset=['DATA_COMPLETA'])
df = df.drop_duplicates(subset=['DATA_COMPLETA'])
df['T2M_MED'] = (df['T2M_MAX'] + df['T2M_MIN']) / 2

# === 2. Normalizacao + Imputacao ===
variaveis = ['T2M_MAX', 'T2M_MIN', 'T2M_MED']
dados_norm = StandardScaler().fit_transform(df[variaveis])
df[variaveis] = IterativeSVD(rank=2).fit_transform(df[variaveis])

# === 3. Climatologia 1991-2020 ===
df['DOY'] = df['DATA_COMPLETA'].dt.dayofyear
base_normal = df[(df['YEAR'] >= 1991) & (df['YEAR'] <= 2020)]
climatologia = base_normal.groupby('DOY').agg(
    Normal_Tmax=('T2M_MAX', 'mean'),
    Desvio_Tmax=('T2M_MAX', 'std'),
    Percentil90_Tmax=('T2M_MAX', lambda x: np.percentile(x, 90)),
    Percentil10_Tmax=('T2M_MAX', lambda x: np.percentile(x, 10)),

    Normal_Tmin=('T2M_MIN', 'mean'),
    Desvio_Tmin=('T2M_MIN', 'std'),
    Percentil90_Tmin=('T2M_MIN', lambda x: np.percentile(x, 90)),
    Percentil10_Tmin=('T2M_MIN', lambda x: np.percentile(x, 10)),

    Normal_Tmed=('T2M_MED', 'mean'),
    Desvio_Tmed=('T2M_MED', 'std')
).reset_index()

df = df.merge(climatologia, on='DOY', how='left')

# === 4. Anomalias e Extremos ===
df['Anomalia_Tmax'] = df['T2M_MAX'] - df['Normal_Tmax']
df['Anomalia_Tmin'] = df['T2M_MIN'] - df['Normal_Tmin']
df['Extremo_Calor_Tmax'] = df['T2M_MAX'] > df['Percentil90_Tmax']
df['Extremo_Frío_Tmin'] = df['T2M_MIN'] < df['Percentil10_Tmin']
df['Evento_Extremo'] = df['Extremo_Calor_Tmax'] | df['Extremo_Frío_Tmin']

# === 5. Exportar resultados parciais ===
df.to_excel(os.path.join(caminho_saida, "Serie_Completa_Tratada.xlsx"), index=False)
climatologia.to_excel(os.path.join(caminho_saida, "Climatologia_1991_2020.xlsx"), index=False)

# === 6. Identificadores ETCCDI ===
df['SU'] = df['T2M_MAX'] > 25

# === 7. Ondas de Calor (WSDI) ===
def identificar_ondas(col_extremo, min_dias=3):
    sequencias = []
    atual = []
    for i, flag in enumerate(df[col_extremo]):
        if flag:
            atual.append(i)
        else:
            if len(atual) >= min_dias:
                sequencias.append(atual)
            atual = []
    if len(atual) >= min_dias:
        sequencias.append(atual)
    return sequencias

ondas_calor = identificar_ondas('Extremo_Calor_Tmax', min_dias=3)
df['WSDI'] = False
for seq in ondas_calor:
    df.loc[seq, 'WSDI'] = True

# === 8. Exportar lista de ondas ===
ondas_df = pd.DataFrame([{ "Inicio": df.loc[seq[0], 'DATA_COMPLETA'],
                           "Fim": df.loc[seq[-1], 'DATA_COMPLETA'],
                           "Duracao": len(seq) } for seq in ondas_calor])
ondas_df.to_excel(os.path.join(caminho_saida, "Ondas_Calor_Maputo.xlsx"), index=False)

# === 9. Calcular TX90p e TN10p anuais ===
dias_ano = df.groupby('YEAR').size()
tx90p = df.groupby('YEAR')['Extremo_Calor_Tmax'].sum() / dias_ano * 100
tn10p = df.groupby('YEAR')['Extremo_Frío_Tmin'].sum() / dias_ano * 100

# === 10. Exportar indices TX90p / TN10p ===
tx90p.to_frame(name='TX90p (%)').join(tn10p.to_frame(name='TN10p (%)')).to_excel(
    os.path.join(caminho_saida, "Indices_TX90p_TN10p.xlsx"))

print("\n✅ Código executado com sucesso!")
print(f"Arquivos salvos em: {caminho_saida}")


c:\Users\Dell\anaconda3\Lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[IterativeSVD] Iter 1: observed MAE=0.930593
[IterativeSVD] Iter 2: observed MAE=0.000000
[IterativeSVD] Iter 3: observed MAE=0.000000
[IterativeSVD] Iter 4: observed MAE=0.000000
[IterativeSVD] Iter 5: observed MAE=0.000000
[IterativeSVD] Iter 6: observed MAE=0.000000
[IterativeSVD] Iter 7: observed MAE=0.000000
[IterativeSVD] Iter 8: observed MAE=0.000000
[IterativeSVD] Iter 9: observed MAE=0.000000
[IterativeSVD] Iter 10: observed MAE=0.000000
[IterativeSVD] Iter 11: observed MAE=0.000000
[IterativeSVD] Iter 12: observed MAE=0.000000
[IterativeSVD] Iter 13: observed MAE=0.000000
[IterativeSVD] Iter 14: observed MAE=0.000000
[IterativeSVD] Iter 15: observed MAE=0.000000
[IterativeSVD] Iter 16: observed MAE=0.000000
[IterativeSVD] Iter 17: observed MAE=0.000000
[IterativeSVD] Iter 18: observed MAE=0.000000
[IterativeSVD] Iter 19: observed MAE=0.000000
[IterativeSVD] Iter 20: observed MAE=0.000000
[IterativeSVD] Iter 21: observed MAE=0.000000
[IterativeSVD] Iter 22: observed MAE=0.0000

In [14]:
# =========================================
# ANALISE DE EXTREMOS TERMICOS - PADRAO WMO/IPCC/FAO/INPE
# =========================================

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import genextreme
from sklearn.preprocessing import StandardScaler
from fancyimpute import IterativeSVD
import warnings
warnings.filterwarnings('ignore')

# === CONFIGURACOES ===
caminho_arquivo = "C:/Python/Waves/maputoo.xlsx"
caminho_saida = "C:/Users/Dell/OneDrive/Rstudio/CONGRESSOJOH/EXTREMOS"
os.makedirs(caminho_saida, exist_ok=True)

# === 1. Carregar dados ===
df = pd.read_excel(caminho_arquivo)
df.columns = ['YEAR', 'DIA_MES', 'T2M_MAX', 'T2M_MIN']
df['DIA_MES'] = pd.to_datetime(df['DIA_MES'])

def corrigir_ano(row):
    try:
        return row['DIA_MES'].replace(year=int(row['YEAR']))
    except ValueError:
        return row['DIA_MES'].replace(day=28, year=int(row['YEAR'])) if row['DIA_MES'].month == 2 and row['DIA_MES'].day == 29 else pd.NaT

df['DATA_COMPLETA'] = df.apply(corrigir_ano, axis=1)
df = df.dropna(subset=['DATA_COMPLETA'])
df = df.drop_duplicates(subset=['DATA_COMPLETA'])
df['T2M_MED'] = (df['T2M_MAX'] + df['T2M_MIN']) / 2

# === 2. Normalizacao + Imputacao ===
variaveis = ['T2M_MAX', 'T2M_MIN', 'T2M_MED']
dados_norm = StandardScaler().fit_transform(df[variaveis])
df[variaveis] = IterativeSVD(rank=2).fit_transform(df[variaveis])

# === 3. Climatologia 1991-2020 ===
df['DOY'] = df['DATA_COMPLETA'].dt.dayofyear
base_normal = df[(df['YEAR'] >= 1991) & (df['YEAR'] <= 2020)]
climatologia = base_normal.groupby('DOY').agg(
    Normal_Tmax=('T2M_MAX', 'mean'),
    Desvio_Tmax=('T2M_MAX', 'std'),
    Percentil90_Tmax=('T2M_MAX', lambda x: np.percentile(x, 90)),
    Percentil10_Tmax=('T2M_MAX', lambda x: np.percentile(x, 10)),

    Normal_Tmin=('T2M_MIN', 'mean'),
    Desvio_Tmin=('T2M_MIN', 'std'),
    Percentil90_Tmin=('T2M_MIN', lambda x: np.percentile(x, 90)),
    Percentil10_Tmin=('T2M_MIN', lambda x: np.percentile(x, 10)),

    Normal_Tmed=('T2M_MED', 'mean'),
    Desvio_Tmed=('T2M_MED', 'std')
).reset_index()

df = df.merge(climatologia, on='DOY', how='left')

# === 4. Anomalias e Extremos ===
df['Anomalia_Tmax'] = df['T2M_MAX'] - df['Normal_Tmax']
df['Anomalia_Tmin'] = df['T2M_MIN'] - df['Normal_Tmin']
df['Extremo_Calor_Tmax'] = df['T2M_MAX'] > df['Percentil90_Tmax']
df['Extremo_Frío_Tmin'] = df['T2M_MIN'] < df['Percentil10_Tmin']
df['Evento_Extremo'] = df['Extremo_Calor_Tmax'] | df['Extremo_Frío_Tmin']

# === 5. Exportar resultados parciais ===
df.to_excel(os.path.join(caminho_saida, "Serie_Completa_Tratada.xlsx"), index=False)
climatologia.to_excel(os.path.join(caminho_saida, "Climatologia_1991_2020.xlsx"), index=False)

# === 6. Identificadores ETCCDI ===
df['SU'] = df['T2M_MAX'] > 25

# === 7. Ondas de Calor (WSDI) ===
def identificar_ondas(col_extremo, min_dias=3):
    sequencias = []
    atual = []
    for i, flag in enumerate(df[col_extremo]):
        if flag:
            atual.append(i)
        else:
            if len(atual) >= min_dias:
                sequencias.append(atual)
            atual = []
    if len(atual) >= min_dias:
        sequencias.append(atual)
    return sequencias

ondas_calor = identificar_ondas('Extremo_Calor_Tmax', min_dias=3)
df['WSDI'] = False
for seq in ondas_calor:
    df.loc[seq, 'WSDI'] = True

# === 8. Ondas de Frio (CSDI) ===
ondas_frio = identificar_ondas('Extremo_Frío_Tmin', min_dias=3)
df['CSDI'] = False
for seq in ondas_frio:
    df.loc[seq, 'CSDI'] = True

# === 9. Exportar lista de ondas ===
ondas_df = pd.DataFrame([{ "Inicio": df.loc[seq[0], 'DATA_COMPLETA'],
                           "Fim": df.loc[seq[-1], 'DATA_COMPLETA'],
                           "Duracao": len(seq),
                           "Tipo": "Calor" } for seq in ondas_calor] +
                         [{ "Inicio": df.loc[seq[0], 'DATA_COMPLETA'],
                           "Fim": df.loc[seq[-1], 'DATA_COMPLETA'],
                           "Duracao": len(seq),
                           "Tipo": "Frio" } for seq in ondas_frio])
ondas_df.to_excel(os.path.join(caminho_saida, "Ondas_Calor_Frio_Maputo.xlsx"), index=False)

# === 10. Calcular TX90p e TN10p anuais ===
dias_ano = df.groupby('YEAR').size()
tx90p = df.groupby('YEAR')['Extremo_Calor_Tmax'].sum() / dias_ano * 100
tn10p = df.groupby('YEAR')['Extremo_Frío_Tmin'].sum() / dias_ano * 100

# === 11. Exportar indices TX90p / TN10p ===
tx90p.to_frame(name='TX90p (%)').join(tn10p.to_frame(name='TN10p (%)')).to_excel(
    os.path.join(caminho_saida, "Indices_TX90p_TN10p.xlsx"))

# === 12. Agrupamento Sazonal ===
df['MES'] = df['DATA_COMPLETA'].dt.month

def estacao(m):
    if m in [12, 1, 2]: return 'DJF'
    if m in [3, 4, 5]: return 'MAM'
    if m in [6, 7, 8]: return 'JJA'
    return 'SON'

df['SAZONAL'] = df['MES'].apply(estacao)
sazonal = df.groupby(['YEAR', 'SAZONAL']).agg(
    Dias_Calor=('Extremo_Calor_Tmax', 'sum'),
    Dias_Frio=('Extremo_Frío_Tmin', 'sum'),
    Media_Anom_Tmax=('Anomalia_Tmax', 'mean'),
    Media_Anom_Tmin=('Anomalia_Tmin', 'mean')
).reset_index()
sazonal.to_excel(os.path.join(caminho_saida, "Resumo_Sazonal_Extremos.xlsx"), index=False)

print("\n✅ Código completo com WMO/IPCC/FAO atualizado!")
print(f"Arquivos salvos em: {caminho_saida}")


c:\Users\Dell\anaconda3\Lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[IterativeSVD] Iter 1: observed MAE=0.930593
[IterativeSVD] Iter 2: observed MAE=0.000000
[IterativeSVD] Iter 3: observed MAE=0.000000
[IterativeSVD] Iter 4: observed MAE=0.000000
[IterativeSVD] Iter 5: observed MAE=0.000000
[IterativeSVD] Iter 6: observed MAE=0.000000
[IterativeSVD] Iter 7: observed MAE=0.000000
[IterativeSVD] Iter 8: observed MAE=0.000000
[IterativeSVD] Iter 9: observed MAE=0.000000
[IterativeSVD] Iter 10: observed MAE=0.000000
[IterativeSVD] Iter 11: observed MAE=0.000000
[IterativeSVD] Iter 12: observed MAE=0.000000
[IterativeSVD] Iter 13: observed MAE=0.000000
[IterativeSVD] Iter 14: observed MAE=0.000000
[IterativeSVD] Iter 15: observed MAE=0.000000
[IterativeSVD] Iter 16: observed MAE=0.000000
[IterativeSVD] Iter 17: observed MAE=0.000000
[IterativeSVD] Iter 18: observed MAE=0.000000
[IterativeSVD] Iter 19: observed MAE=0.000000
[IterativeSVD] Iter 20: observed MAE=0.000000
[IterativeSVD] Iter 21: observed MAE=0.000000
[IterativeSVD] Iter 22: observed MAE=0.0000

In [15]:
# =========================================
# ANALISE DE EXTREMOS TERMICOS - PADRAO WMO/IPCC/FAO/INPE
# =========================================

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import genextreme
from sklearn.preprocessing import StandardScaler
from fancyimpute import IterativeSVD
import warnings
warnings.filterwarnings('ignore')

# === CONFIGURACOES ===
caminho_arquivo = "C:/Python/Waves/maputoo.xlsx"
caminho_saida = "C:/Users/Dell/OneDrive/Rstudio/CONGRESSOJOH/EXTREMOS"
os.makedirs(caminho_saida, exist_ok=True)

# === 1. Carregar dados ===
df = pd.read_excel(caminho_arquivo)
df.columns = ['YEAR', 'DIA_MES', 'T2M_MAX', 'T2M_MIN']
df['DIA_MES'] = pd.to_datetime(df['DIA_MES'])

def corrigir_ano(row):
    try:
        return row['DIA_MES'].replace(year=int(row['YEAR']))
    except ValueError:
        return row['DIA_MES'].replace(day=28, year=int(row['YEAR'])) if row['DIA_MES'].month == 2 and row['DIA_MES'].day == 29 else pd.NaT

df['DATA_COMPLETA'] = df.apply(corrigir_ano, axis=1)
df = df.dropna(subset=['DATA_COMPLETA'])
df = df.drop_duplicates(subset=['DATA_COMPLETA'])
df['T2M_MED'] = (df['T2M_MAX'] + df['T2M_MIN']) / 2

# === 2. Normalizacao + Imputacao ===
variaveis = ['T2M_MAX', 'T2M_MIN', 'T2M_MED']
dados_norm = StandardScaler().fit_transform(df[variaveis])
df[variaveis] = IterativeSVD(rank=2).fit_transform(df[variaveis])

# === 3. Climatologia 1991-2020 ===
df['DOY'] = df['DATA_COMPLETA'].dt.dayofyear
base_normal = df[(df['YEAR'] >= 1991) & (df['YEAR'] <= 2020)]
climatologia = base_normal.groupby('DOY').agg(
    Normal_Tmax=('T2M_MAX', 'mean'),
    Desvio_Tmax=('T2M_MAX', 'std'),
    Percentil90_Tmax=('T2M_MAX', lambda x: np.percentile(x, 90)),
    Percentil10_Tmax=('T2M_MAX', lambda x: np.percentile(x, 10)),

    Normal_Tmin=('T2M_MIN', 'mean'),
    Desvio_Tmin=('T2M_MIN', 'std'),
    Percentil90_Tmin=('T2M_MIN', lambda x: np.percentile(x, 90)),
    Percentil10_Tmin=('T2M_MIN', lambda x: np.percentile(x, 10)),

    Normal_Tmed=('T2M_MED', 'mean'),
    Desvio_Tmed=('T2M_MED', 'std')
).reset_index()

df = df.merge(climatologia, on='DOY', how='left')

# === 4. Anomalias e Extremos ===
df['Anomalia_Tmax'] = df['T2M_MAX'] - df['Normal_Tmax']
df['Anomalia_Tmin'] = df['T2M_MIN'] - df['Normal_Tmin']
df['Extremo_Calor_Tmax'] = df['T2M_MAX'] > df['Percentil90_Tmax']
df['Extremo_Frío_Tmin'] = df['T2M_MIN'] < df['Percentil10_Tmin']
df['Evento_Extremo'] = df['Extremo_Calor_Tmax'] | df['Extremo_Frío_Tmin']

# === 5. Exportar resultados parciais ===
df.to_excel(os.path.join(caminho_saida, "Serie_Completa_Tratada.xlsx"), index=False)
climatologia.to_excel(os.path.join(caminho_saida, "Climatologia_1991_2020.xlsx"), index=False)

# === 6. Identificadores ETCCDI ===
df['SU'] = df['T2M_MAX'] > 25

# === 7. Ondas de Calor (WSDI) ===
def identificar_ondas(col_extremo, min_dias=3):
    sequencias = []
    atual = []
    for i, flag in enumerate(df[col_extremo]):
        if flag:
            atual.append(i)
        else:
            if len(atual) >= min_dias:
                sequencias.append(atual)
            atual = []
    if len(atual) >= min_dias:
        sequencias.append(atual)
    return sequencias

ondas_calor = identificar_ondas('Extremo_Calor_Tmax', min_dias=3)
df['WSDI'] = False
for seq in ondas_calor:
    df.loc[seq, 'WSDI'] = True

# === 8. Ondas de Frio (CSDI) ===
ondas_frio = identificar_ondas('Extremo_Frío_Tmin', min_dias=3)
df['CSDI'] = False
for seq in ondas_frio:
    df.loc[seq, 'CSDI'] = True

# === 9. Exportar lista de ondas ===
ondas_df = pd.DataFrame([{ "Inicio": df.loc[seq[0], 'DATA_COMPLETA'],
                           "Fim": df.loc[seq[-1], 'DATA_COMPLETA'],
                           "Duracao": len(seq),
                           "Tipo": "Calor" } for seq in ondas_calor] +
                         [{ "Inicio": df.loc[seq[0], 'DATA_COMPLETA'],
                           "Fim": df.loc[seq[-1], 'DATA_COMPLETA'],
                           "Duracao": len(seq),
                           "Tipo": "Frio" } for seq in ondas_frio])
ondas_df.to_excel(os.path.join(caminho_saida, "Ondas_Calor_Frio_Maputo.xlsx"), index=False)

# === 10. Calcular TX90p e TN10p anuais ===
dias_ano = df.groupby('YEAR').size()
tx90p = df.groupby('YEAR')['Extremo_Calor_Tmax'].sum() / dias_ano * 100
tn10p = df.groupby('YEAR')['Extremo_Frío_Tmin'].sum() / dias_ano * 100

# === 11. Exportar indices TX90p / TN10p ===
tx90p.to_frame(name='TX90p (%)').join(tn10p.to_frame(name='TN10p (%)')).to_excel(
    os.path.join(caminho_saida, "Indices_TX90p_TN10p.xlsx"))

# === 12. Agrupamento Sazonal ===
df['MES'] = df['DATA_COMPLETA'].dt.month

def estacao(m):
    if m in [12, 1, 2]: return 'DJF'
    if m in [3, 4, 5]: return 'MAM'
    if m in [6, 7, 8]: return 'JJA'
    return 'SON'

df['SAZONAL'] = df['MES'].apply(estacao)
sazonal = df.groupby(['YEAR', 'SAZONAL']).agg(
    Dias_Calor=('Extremo_Calor_Tmax', 'sum'),
    Dias_Frio=('Extremo_Frío_Tmin', 'sum'),
    Media_Anom_Tmax=('Anomalia_Tmax', 'mean'),
    Media_Anom_Tmin=('Anomalia_Tmin', 'mean')
).reset_index()
sazonal.to_excel(os.path.join(caminho_saida, "Resumo_Sazonal_Extremos.xlsx"), index=False)

# === 13. Graficos cientificos ===
sns.set(style="whitegrid")

# Frequência de ondas de calor e frio
plt.figure(figsize=(12,6))
plt.plot(tx90p.index, tx90p.values, marker='o', color='red', label='TX90p')
plt.plot(tn10p.index, tn10p.values, marker='o', color='blue', label='TN10p')
plt.xlabel("Ano")
plt.ylabel("% Dias")
plt.title("Índices TX90p e TN10p")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(caminho_saida, "Grafico_TX90p_TN10p.png"), dpi=300)
plt.close()

# Ondas de calor e frio - duração
plt.figure(figsize=(12,6))
sns.histplot(ondas_df[ondas_df['Tipo']=="Calor"]['Duracao'], color='orangered', label='Ondas de Calor', kde=False)
sns.histplot(ondas_df[ondas_df['Tipo']=="Frio"]['Duracao'], color='royalblue', label='Ondas de Frio', kde=False)
plt.xlabel("Duração (dias)")
plt.ylabel("Frequência")
plt.title("Distribuição da Duração das Ondas de Calor e Frio")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(caminho_saida, "Distribuicao_Duracao_Ondas.png"), dpi=300)
plt.close()
print("\n✅ Código completo com gráficos salvos em PNG e PDF!")
print(f"Arquivos salvos em: {caminho_saida}")


c:\Users\Dell\anaconda3\Lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[IterativeSVD] Iter 1: observed MAE=0.930593
[IterativeSVD] Iter 2: observed MAE=0.000000
[IterativeSVD] Iter 3: observed MAE=0.000000
[IterativeSVD] Iter 4: observed MAE=0.000000
[IterativeSVD] Iter 5: observed MAE=0.000000
[IterativeSVD] Iter 6: observed MAE=0.000000
[IterativeSVD] Iter 7: observed MAE=0.000000
[IterativeSVD] Iter 8: observed MAE=0.000000
[IterativeSVD] Iter 9: observed MAE=0.000000
[IterativeSVD] Iter 10: observed MAE=0.000000
[IterativeSVD] Iter 11: observed MAE=0.000000
[IterativeSVD] Iter 12: observed MAE=0.000000
[IterativeSVD] Iter 13: observed MAE=0.000000
[IterativeSVD] Iter 14: observed MAE=0.000000
[IterativeSVD] Iter 15: observed MAE=0.000000
[IterativeSVD] Iter 16: observed MAE=0.000000
[IterativeSVD] Iter 17: observed MAE=0.000000
[IterativeSVD] Iter 18: observed MAE=0.000000
[IterativeSVD] Iter 19: observed MAE=0.000000
[IterativeSVD] Iter 20: observed MAE=0.000000
[IterativeSVD] Iter 21: observed MAE=0.000000
[IterativeSVD] Iter 22: observed MAE=0.0000

In [16]:
# =========================================
# ANALISE DE EXTREMOS TERMICOS - PADRAO WMO/IPCC/FAO/INPE
# =========================================

# === 14. Painéis científicos (A, B, C) ===
# Painel A: Climatologia e Anomalias
fig, axs = plt.subplots(2, 1, figsize=(14,10), sharex=True)
axs[0].plot(climatologia['DOY'], climatologia['Normal_Tmax'], label='Tmax Média', color='red')
axs[0].plot(climatologia['DOY'], climatologia['Normal_Tmin'], label='Tmin Média', color='blue')
axs[0].fill_between(climatologia['DOY'],
                    climatologia['Normal_Tmax'] - 2 * climatologia['Desvio_Tmax'],
                    climatologia['Normal_Tmax'] + 2 * climatologia['Desvio_Tmax'],
                    color='red', alpha=0.3)
axs[0].fill_between(climatologia['DOY'],
                    climatologia['Normal_Tmin'] - 2 * climatologia['Desvio_Tmin'],
                    climatologia['Normal_Tmin'] + 2 * climatologia['Desvio_Tmin'],
                    color='blue', alpha=0.3)
axs[0].set_ylabel('Temperatura (°C)')
axs[0].set_title('Painel A1 - Climatologia de Tmax e Tmin')
axs[0].legend()
axs[0].grid(True)

axs[1].plot(df['DATA_COMPLETA'], df['Anomalia_Tmax'], label='Anomalia Tmax', color='darkred')
axs[1].plot(df['DATA_COMPLETA'], df['Anomalia_Tmin'], label='Anomalia Tmin', color='darkblue')
axs[1].axhline(0, color='black', linestyle='--')
axs[1].set_ylabel('Anomalia (°C)')
axs[1].set_title('Painel A2 - Anomalias de Temperatura')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(caminho_saida, "Painel_A_Climatologia_Anomalias.png"), dpi=300)
plt.close()

# Painel B: Frequência e Severidade
fig, axs = plt.subplots(3, 1, figsize=(14,12))
axs[0].bar(tx90p.index, tx90p.values, color='orangered', label='TX90p')
axs[0].bar(tn10p.index, -tn10p.values, color='royalblue', label='TN10p')
axs[0].axhline(0, color='black')
axs[0].set_title('Painel B1 - Frequência de Extremos de Calor e Frio')
axs[0].set_ylabel('% Dias')
axs[0].legend()
axs[0].grid(True)

# Frequência de eventos severos
freq_severo_calor = df.groupby('YEAR')['Extremo_Calor_Tmax'].sum()
freq_severo_frio = df.groupby('YEAR')['Extremo_Frío_Tmin'].sum()
axs[1].plot(freq_severo_calor.index, freq_severo_calor.values, marker='o', label='Eventos Severos Calor', color='darkred')
axs[1].plot(freq_severo_frio.index, freq_severo_frio.values, marker='o', label='Eventos Severos Frio', color='darkblue')
axs[1].set_title('Painel B2 - Eventos Severos')
axs[1].set_ylabel('Nº de dias')
axs[1].legend()
axs[1].grid(True)

# Tendência linear
from scipy.stats import linregress
trend_tx = linregress(tx90p.index, tx90p.values)
trend_tn = linregress(tn10p.index, tn10p.values)
axs[2].plot(tx90p.index, tx90p.values, color='red', label='TX90p')
axs[2].plot(tx90p.index, trend_tx.intercept + trend_tx.slope * tx90p.index, 'r--', label=f'Tendência TX90p ({trend_tx.slope:.2f}/ano)')
axs[2].plot(tn10p.index, tn10p.values, color='blue', label='TN10p')
axs[2].plot(tn10p.index, trend_tn.intercept + trend_tn.slope * tn10p.index, 'b--', label=f'Tendência TN10p ({trend_tn.slope:.2f}/ano)')
axs[2].set_title('Painel B3 - Tendência Linear dos Índices')
axs[2].set_ylabel('% Dias')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(caminho_saida, "Painel_B_Extremos_Indices_Tendencia.png"), dpi=300)
plt.close()

# Painel C: Duração e Intensidade
dur_calor = ondas_df[ondas_df['Tipo']=="Calor"]['Duracao']
dur_frio = ondas_df[ondas_df['Tipo']=="Frio"]['Duracao']
fig, axs = plt.subplots(2, 1, figsize=(14,10))
axs[0].hist(dur_calor, bins=range(1, max(dur_calor)+2), alpha=0.6, label='Ondas de Calor', color='orange')
axs[0].hist(dur_frio, bins=range(1, max(dur_frio)+2), alpha=0.6, label='Ondas de Frio', color='blue')
axs[0].set_title('Painel C1 - Duração das Ondas')
axs[0].set_xlabel('Dias consecutivos')
axs[0].set_ylabel('Frequência')
axs[0].legend()
axs[0].grid(True)

# Intensidade média por ano (simplificado)
intensidade_calor = df[df['Extremo_Calor_Tmax']]['T2M_MAX'] - df[df['Extremo_Calor_Tmax']]['Percentil90_Tmax']
intensidade_frio = df[df['Extremo_Frío_Tmin']]['Percentil10_Tmin'] - df[df['Extremo_Frío_Tmin']]['T2M_MIN']
int_ano_calor = intensidade_calor.groupby(df.loc[df['Extremo_Calor_Tmax'], 'YEAR']).mean()
int_ano_frio = intensidade_frio.groupby(df.loc[df['Extremo_Frío_Tmin'], 'YEAR']).mean()
axs[1].plot(int_ano_calor.index, int_ano_calor.values, marker='o', label='Intensidade Calor', color='darkorange')
axs[1].plot(int_ano_frio.index, int_ano_frio.values, marker='o', label='Intensidade Frio', color='navy')
axs[1].set_title('Painel C2 - Intensidade Média Anual')
axs[1].set_ylabel('°C acima/abaixo limite')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(caminho_saida, "Painel_C_Duracao_Intensidade.png"), dpi=300)
plt.close()

print("\n✅ Painéis científicos A, B e C gerados e salvos com sucesso!")



✅ Painéis científicos A, B e C gerados e salvos com sucesso!


In [17]:
# =========================================
# ANALISE DE EXTREMOS TERMICOS - PADRAO WMO/IPCC/FAO/INPE
# =========================================

print("\n✅ Painéis científicos A, B e C gerados e salvos com sucesso!")

# === 15. Painel D - Variações Sazonais ===
fig, axs = plt.subplots(2, 2, figsize=(16, 10))

saz = ['DJF', 'MAM', 'JJA', 'SON']
cores = ['firebrick', 'forestgreen', 'dodgerblue', 'darkorange']

for i, sazonalidade in enumerate(saz):
    dados = sazonal[sazonal['SAZONAL'] == sazonalidade]
    ax = axs.flat[i]
    ax.plot(dados['YEAR'], dados['Dias_Calor'], marker='o', color=cores[i], label='Calor')
    ax.plot(dados['YEAR'], dados['Dias_Frio'], marker='s', color='black', label='Frio')
    ax.set_title(f"Painel D{i+1} - {sazonalidade}")
    ax.set_xlabel('Ano')
    ax.set_ylabel('Dias extremos')
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(caminho_saida, "Painel_D_Sazonalidade_Extremos.png"), dpi=300)
plt.close()

# === 16. Exportar PDF único com todos os gráficos ===
from matplotlib.backends.backend_pdf import PdfPages

pdf_final = os.path.join(caminho_saida, "Relatorio_Visual_Extremos_Maputo.pdf")
figuras = [
    "Painel_A_Climatologia_Anomalias",
    "Painel_B_Extremos_Indices_Tendencia",
    "Painel_C_Duracao_Intensidade",
    "Painel_D_Sazonalidade_Extremos"
]

with PdfPages(pdf_final) as pdf:
    for nome in figuras:
        fig = plt.imread(os.path.join(caminho_saida, f"{nome}.png"))
        plt.figure(figsize=(12, 8))
        plt.imshow(fig)
        plt.axis('off')
        pdf.savefig()
        plt.close()

print("\n✅ Painel D (sazonalidade) salvo com sucesso!")
print(f"📄 PDF consolidado salvo em: {pdf_final}")



✅ Painéis científicos A, B e C gerados e salvos com sucesso!

✅ Painel D (sazonalidade) salvo com sucesso!
📄 PDF consolidado salvo em: C:/Users/Dell/OneDrive/Rstudio/CONGRESSOJOH/EXTREMOS\Relatorio_Visual_Extremos_Maputo.pdf


In [18]:
# =========================================
# ANALISE DE EXTREMOS TERMICOS - PADRAO WMO/IPCC/FAO/INPE
# =========================================

print("\n✅ Painéis científicos A, B e C gerados e salvos com sucesso!")

# === 15. Painel D - Variações Sazonais ===
fig, axs = plt.subplots(2, 2, figsize=(16, 10))

saz = ['DJF', 'MAM', 'JJA', 'SON']
cores = ['firebrick', 'forestgreen', 'dodgerblue', 'darkorange']

for i, sazonalidade in enumerate(saz):
    dados = sazonal[sazonal['SAZONAL'] == sazonalidade]
    ax = axs.flat[i]
    ax.plot(dados['YEAR'], dados['Dias_Calor'], marker='o', color=cores[i], label='Calor')
    ax.plot(dados['YEAR'], dados['Dias_Frio'], marker='s', color='black', label='Frio')
    ax.set_title(f"Painel D{i+1} - {sazonalidade}")
    ax.set_xlabel('Ano')
    ax.set_ylabel('Dias extremos')
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(caminho_saida, "Painel_D_Sazonalidade_Extremos.png"), dpi=300)
plt.close()

# === 16. Exportar PDF único com todos os gráficos ===
from matplotlib.backends.backend_pdf import PdfPages

pdf_final = os.path.join(caminho_saida, "Relatorio_Visual_Extremos_Maputo.pdf")
figuras = [
    "Painel_A_Climatologia_Anomalias",
    "Painel_B_Extremos_Indices_Tendencia",
    "Painel_C_Duracao_Intensidade",
    "Painel_D_Sazonalidade_Extremos"
]

with PdfPages(pdf_final) as pdf:
    for nome in figuras:
        fig = plt.imread(os.path.join(caminho_saida, f"{nome}.png"))
        plt.figure(figsize=(12, 8))
        plt.imshow(fig)
        plt.axis('off')
        pdf.savefig()
        plt.close()

print("\n✅ Painel D (sazonalidade) salvo com sucesso!")
print(f"📄 PDF consolidado salvo em: {pdf_final}")

# === 17. Índices ETCCDI adicionais (corrigido) ===
group = df.groupby('YEAR')
etccdi = group.agg(
    TXx=('T2M_MAX', 'max'),
    TXn=('T2M_MAX', 'min'),
    TNx=('T2M_MIN', 'max'),
    TNn=('T2M_MIN', 'min'),
    FD=('T2M_MIN', lambda x: (x < 0).sum()),
    TR=('T2M_MIN', lambda x: (x > 20).sum()),
    ID=('T2M_MAX', lambda x: (x < 0).sum())
).reset_index()

# Calcular DTR separadamente
tmax_mean = group['T2M_MAX'].mean()
tmin_mean = group['T2M_MIN'].mean()
etccdi['DTR'] = tmax_mean.values - tmin_mean.values

# Exportar
etccdi.to_excel(os.path.join(caminho_saida, "Indices_ETCCDI_Adicionais.xlsx"), index=False)
print("\n✅ Índices adicionais ETCCDI salvos com sucesso!")



✅ Painéis científicos A, B e C gerados e salvos com sucesso!

✅ Painel D (sazonalidade) salvo com sucesso!
📄 PDF consolidado salvo em: C:/Users/Dell/OneDrive/Rstudio/CONGRESSOJOH/EXTREMOS\Relatorio_Visual_Extremos_Maputo.pdf

✅ Índices adicionais ETCCDI salvos com sucesso!


In [19]:
# =========================================
# ANALISE DE EXTREMOS TERMICOS - PADRAO WMO/IPCC/FAO/INPE
# =========================================

print("\n✅ Painéis científicos A, B e C gerados e salvos com sucesso!")

# === 15. Painel D - Variações Sazonais ===
fig, axs = plt.subplots(2, 2, figsize=(16, 10))

saz = ['DJF', 'MAM', 'JJA', 'SON']
cores = ['firebrick', 'forestgreen', 'dodgerblue', 'darkorange']

for i, sazonalidade in enumerate(saz):
    dados = sazonal[sazonal['SAZONAL'] == sazonalidade]
    ax = axs.flat[i]
    ax.plot(dados['YEAR'], dados['Dias_Calor'], marker='o', color=cores[i], label='Calor')
    ax.plot(dados['YEAR'], dados['Dias_Frio'], marker='s', color='black', label='Frio')
    ax.set_title(f"Painel D{i+1} - {sazonalidade}")
    ax.set_xlabel('Ano')
    ax.set_ylabel('Dias extremos')
    ax.grid(True)
    ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(caminho_saida, "Painel_D_Sazonalidade_Extremos.png"), dpi=300)
plt.close()

# === 16. Exportar PDF único com todos os gráficos ===
from matplotlib.backends.backend_pdf import PdfPages

pdf_final = os.path.join(caminho_saida, "Relatorio_Visual_Extremos_Maputo.pdf")
figuras = [
    "Painel_A_Climatologia_Anomalias",
    "Painel_B_Extremos_Indices_Tendencia",
    "Painel_C_Duracao_Intensidade",
    "Painel_D_Sazonalidade_Extremos",
    "Painel_ETCCDI_Adicionais"
]

with PdfPages(pdf_final) as pdf:
    for nome in figuras:
        caminho_img = os.path.join(caminho_saida, f"{nome}.png")
        if os.path.exists(caminho_img):
            fig = plt.imread(caminho_img)
            plt.figure(figsize=(12, 8))
            plt.imshow(fig)
            plt.axis('off')
            pdf.savefig()
            plt.close()

print("\n✅ Painel D (sazonalidade) salvo com sucesso!")
print(f"📄 PDF consolidado salvo em: {pdf_final}")

# === 17. Índices ETCCDI adicionais (corrigido) ===
group = df.groupby('YEAR')

# Calcular separadamente as médias de Tmax e Tmin para DTR
tmax_mean = group['T2M_MAX'].mean()
tmin_mean = group['T2M_MIN'].mean()
dtr = tmax_mean - tmin_mean

# Calcular os demais índices
etccdi = group.agg(
    TXx=('T2M_MAX', 'max'),
    TXn=('T2M_MAX', 'min'),
    TNx=('T2M_MIN', 'max'),
    TNn=('T2M_MIN', 'min'),
    FD=('T2M_MIN', lambda x: (x < 0).sum()),
    TR=('T2M_MIN', lambda x: (x > 20).sum()),
    ID=('T2M_MAX', lambda x: (x < 0).sum())
).reset_index()

# Adicionar a coluna DTR
etccdi['DTR'] = dtr.values

# Salvar como Excel
etccdi.to_excel(os.path.join(caminho_saida, "Indices_ETCCDI_Adicionais.xlsx"), index=False)
print("\n✅ Índices adicionais ETCCDI salvos com sucesso!")

# === 18. Gráficos dos índices ETCCDI adicionais ===
fig, axs = plt.subplots(4, 2, figsize=(16, 20))
indices = ['TXx', 'TXn', 'TNx', 'TNn', 'DTR', 'FD', 'TR', 'ID']
cores = ['darkred', 'indianred', 'darkblue', 'royalblue', 'purple', 'grey', 'orange', 'black']

for i, ind in enumerate(indices):
    ax = axs.flat[i]
    ax.plot(etccdi['YEAR'], etccdi[ind], marker='o', color=cores[i])
    ax.set_title(ind)
    ax.set_xlabel('Ano')
    ax.set_ylabel(ind)
    ax.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(caminho_saida, "Painel_ETCCDI_Adicionais.png"), dpi=300)
plt.close()

print("\n✅ Gráficos dos índices ETCCDI adicionais gerados com sucesso!")


✅ Painéis científicos A, B e C gerados e salvos com sucesso!

✅ Painel D (sazonalidade) salvo com sucesso!
📄 PDF consolidado salvo em: C:/Users/Dell/OneDrive/Rstudio/CONGRESSOJOH/EXTREMOS\Relatorio_Visual_Extremos_Maputo.pdf

✅ Índices adicionais ETCCDI salvos com sucesso!

✅ Gráficos dos índices ETCCDI adicionais gerados com sucesso!
